In [2]:
import serial
import time
import pandas as pd
import plotly.express as px


In [3]:
ser = serial.Serial('COM9', 115200, timeout=1)

In [7]:
def display_Graphs(expected_distance):
    ser.reset_input_buffer()
    buffer = []
    while True:
        data = ser.readline().decode('utf-8').strip()
        if data:
            buffer.append(data)
            if len(buffer) > 2000:
                break
        else:
            time.sleep(0.1)
    df = pd.DataFrame(buffer, columns=['raw_data'])
    # display(df)
    #convert to numeric
    df['raw_data'] = pd.to_numeric(df['raw_data'])
    #if val is bigger than 1000, it is probably an error	
    fig = px.line(df, y='raw_data')
    fig.show()

    def convertToDistance(raw_data):
        return raw_data * 0.017

    df['distance'] = df['raw_data'].apply(convertToDistance)


    fig = px.line(df, y='distance')
    fig.show()

    fig2 = px.histogram(df, x='distance')
    fig2.show()
    #save to csv
    df.to_csv('data_{}.csv'.format(expected_distance), index=False)

In [5]:
display_Graphs(100) # 10 cm

In [6]:
display_Graphs(50) # 50 cm

In [8]:
display_Graphs(20) # 20 cm

In [9]:
display_Graphs(10) # 10 cm

In [10]:
display_Graphs(2) # 2 cm

In [12]:
display_Graphs(1) #minimun break down :c

In [16]:
distance = 100
raw_val = distance / 0.017
print(raw_val)

5882.35294117647


In [8]:
display_Graphs(1000) # 100 cm